In [2]:
import os, logging
import requests, json
import re, sys, subprocess, time

LOG = logging.getLogger()

EXCEPTION_COMMAND = """osascript -e 'tell app "Finder" to display dialog "{}"'"""
BATT_INFO_COMMAND = "pmset -g batt"
LED_JSON_URL = "http://192.168.0.140/json/state"
LED_PRESET_URL = "http://192.168.0.140/win&PL={}"
TEMP_STORAGE_DIR = "/private/tmp/wled_subprocess/{}"
CONFIG_FILE_PATH = TEMP_STORAGE_DIR.format("smart_desk.json")

LED_PRESET_MAP = {
    "charging" : 6,
    "charged"  : 2,
    "low battery" : 5}

def call_err_and_quit(err):
    command = EXCEPTION_COMMAND.format(err)
    subprocess.run([command], shell=True)
    sys.exit()

def get_battery_data():
    LOG.info("Getting battery information")
    batt_info = re.findall("\\\t(.*)", subprocess.run([BATT_INFO_COMMAND], shell=True, capture_output=True, text=True).stdout)
    if (len(batt_info) != 1
        or len(batt_info[0].split(";")) != 3
        or "%" not in batt_info[0].split(";")[0]):

        err = f"""Invalid results for `pmset -g batt`: \n\n{out.replace('"', "").replace("'", "")}"""
        call_err_and_quit(err)

    batt_info  = batt_info[0].split(";")
    batt_data = {
        "level"  : int(batt_info[0].replace("%", "")),
        "status" : batt_info[1].strip()}
    LOG.info(f"Battery info : {json.dumps(batt_data, indent=2)}")
    return batt_data


def get_curr_state():
    header = {
        "content-type" : "application/json"}
    return requests.get(LED_JSON_URL, headers=header).json()


def set_led_preset(preset_num):
    assert isinstance(preset_num, int)
    requests.post(LED_PRESET_URL.format(preset_num))
    return True


def set_led_state(state):
    header = {
        "content-type" : "application/json"}
    requests.post(LED_JSON_URL, headers=header, json=state).json()
    return True


def save_config_file(config):
    # create dir if unavailable
    if not os.path.isdir(TEMP_STORAGE_DIR.format("")):
        os.mkdir(TEMP_STORAGE_DIR.format(""))

    with open(CONFIG_FILE_PATH, "w") as f:
        f.write(json.dumps(config, indent=2))
    return True


def read_from_config():
    # create dir if unavailable
    if not os.path.isfile(CONFIG_FILE_PATH):
        return False

    with open(CONFIG_FILE_PATH, "r") as f:
        a = f.read(CONFIG_FILE_PATH)
        config = json.loads(a)

    if not config:
        return False

    return config


def process_state(battery_info, config):

    if config["state_altered"]:
        if (
            config["state_altered"] and
            battery_info["level"] > 30 and
            battery_info["status"] in ["discharging"]
        ):
            set_led_state(config["saved_state"])
            config["state_altered"] = False
            config["charged"]       = False
            config["low_battery"]   = False
            config["saved_state"]   = None

    else:
        LOG.info("Getting LED current state")
        curr_led_state = get_curr_state()

        if (
            battery_info["level"] > 90  and
            battery_info["status"] in ["charging", "charged"]
        ):
            config["state_altered"] = True
            config["charged"]       = True
            config["low_battery"]   = False
            config["saved_state"]   = curr_led_state
            save_config_file(config)

        elif (
            battery_info["level"] < 30 and
            battery_info["status"] in ["discharging"]
        ):
            config["state_altered"] = True
            config["charged"]       = False
            config["low_battery"]   = True
            config["saved_state"]   = curr_led_state
            save_config_file(config)

    return config



# def main():
LOG.info("Starting script execution...")

LOG.info("Reading configuration")
config = read_from_config()
if not config:
    config = {
        "saved_state"   : None ,
        "state_altered" : False,
        "charged"       : False,
        "low_battery"   : False}


LOG.info("Reading battery stats...")
battery_info  = get_battery_data()

battery_info = {'level': 10, 'status': 'charging'}
config = process_state(battery_info, config)
print(config)






main()

{'saved_state': None, 'state_altered': False, 'charged': False, 'low_battery': False}


NameError: name 'main' is not defined

In [8]:
battery_info

{'level': 10, 'status': 'charging'}